# ML Project

## Import libraries, get data

In [1]:
# Libraries

import pandas as pd
import csv
import json
import glob

In [2]:
m = []
l = []
for i in range (1,13):
    m += [fr"C:\Users\Ram\Downloads\2016_original\2016\On_Time_On_Time_Performance_2016_{i}\On_Time_On_Time_Performance_2016_{i}.csv"]
    l += [fr"C:\Users\Ram\Downloads\2017_original\2017\On_Time_On_Time_Performance_2017_{i}\On_Time_On_Time_Performance_2017_{i}.csv"]
l = m+l
a = ["ATL","CLT","DEN","DFW","EWR","IAH","JFK","LAS","LAX","MCO","MIA","ORD","PHX","SEA"]

In [29]:
columns = ['FlightDate',"Quarter","Origin","Year","Month","DayofMonth","DepTime","DepDel15","CRSDepTime","DepDelayMinutes","OriginAirportID","DestAirportID","ArrTime","CRSArrTime","ArrDel15","ArrDelayMinutes"]


df1 = pd.read_csv(l[0])
df1 = df1[df1["Origin"].isin(a)]
df1 = df1[df1["Dest"].isin(a)]
df1 = df1[df1["DepTime"].notna()]   
df1 = pd.DataFrame(df1, columns=columns)
for i in range(1,len(l)):
    df2 = pd.read_csv(l[i])
    df2 = df2[df2["Dest"].isin(a)]
    df2 = df2[df2["Origin"].isin(a)]
    df2 = df2[df2["DepTime"].notna()]   
    df2 = pd.DataFrame(df2, columns=columns)
    concat_data = pd.concat([df1, df2], ignore_index=True)
    df1 = concat_data



# df1 = df1.iloc[:100]
df1.shape

C:\Users\Ram\AppData\Local\Temp\ipykernel_8792\3932062354.py:4: DtypeWarning: Columns (77) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(l[0])
C:\Users\Ram\AppData\Local\Temp\ipykernel_8792\3932062354.py:10: DtypeWarning: Columns (77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(l[i])
C:\Users\Ram\AppData\Local\Temp\ipykernel_8792\3932062354.py:10: DtypeWarning: Columns (48,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(l[i])
C:\Users\Ram\AppData\Local\Temp\ipykernel_8792\3932062354.py:10: DtypeWarning: Columns (48,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(l[i])
C:\Users\Ram\AppData\Local\Temp\ipykernel_8792\3932062354.py:10: DtypeWarning: Columns (48,77,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(l[i])
C:\Users\Ram\AppD

(1522020, 16)

In [25]:

df1.to_csv('file1.csv')
# Import flight data

# df = pd.read_csv(r'C:\Users\Ram\Downloads\2017_original\2017\On_Time_On_Time_Performance_2017_1\On_Time_On_Time_Performance_2017_1.csv')
# columns = ['FlightDate',"Quarter","Origin","Year","Month","DayofMonth","DepTime","DepDel15","CRSDepTime","DepDelayMinutes","OriginAirportID","DestAirportID","ArrTime","CRSArrTime","ArrDel15","ArrDelayMinutes"]
# df = pd.DataFrame(df, columns=columns)
# df =df[df["DepTime"].notna()]

# df = df.iloc[:100]

In [26]:
ll = ["ATL","CLT","DEN","DFW","EWR","IAH","JFK","LAS","LAX","MCO","MIA","ORD","PHX","SEA"]
# ll = ["JFK"]
all_weather_data = {}
for i in ll:
    city_weather = []
    
    for j in range(1,13):
        city_weather += [f"C:\\Users\\Ram\\Downloads\\{i}\\2016-{j}.json"]
        
    for j in range(1,13):
        city_weather += [f"C:\\Users\\Ram\\Downloads\\{i}\\2017-{j}.json"]
    
    
    new_json = []
    for filename in city_weather:
        with open(filename) as fin1:
            data1 = json.load(fin1)
            data1 = data1['data']['weather']

            new_json += [data1]

    all_weather_data[i] = new_json

with open('all_weather_data.json', 'w+') as f:
    json.dump(all_weather_data, f, indent=4)

In [27]:
df1 = pd.read_csv('file1.csv',header=0)
with open('all_weather_data.json') as f:
    all_weather_data = json.load(f)

In [ ]:
print(all_weather_data['JFK'][1][29]['hourly'][0])

In [30]:
month = {1:31,2:28,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
feb_month = {2016: 29, 2017: 28}
flight_weather_data = []

for index, row in df1.iterrows():
    if df1.at[index,"DepTime"] // 100 == 24 :
        df1.at[index,"DayofMonth"] = df1.at[index,"DayofMonth"] + 1
        df1.at[index,"DepTime"] = df1.at[index,"DepTime"] - 2400
        if df1.at[index,"Month"] == 2:
            month[2] = feb_month[df1.at[index,"Year"]]
        if df1.at[index,"DayofMonth"] > month[df1.at[index,"Month"]]:
            df1.at[index,"DayofMonth"] = 1
            df1.at[index,"Month"] += 1
            if df1.at[index,"Month"] > 12:
                df1.at[index,"Year"] += 1
                df1.at[index,"Month"] = 1
    
    origin_index = df1.at[index,"Origin"]
    dep_month_index = (df1.at[index,"Year"] - 2016) * 12 + df1.at[index,"Month"] - 1
    dep_day_index = int(df1.at[index,"DayofMonth"]) - 1
    dep_time_index = int(df1.at[index,"DepTime"] // 100)
    
    try:
        flight_weather = all_weather_data[origin_index][dep_month_index][dep_day_index]['hourly'][dep_time_index]
        flight_weather_data.append(flight_weather)        
    except:
        print('origin_index', origin_index, 'dep_month_index', dep_month_index, '\tdep_day_index', dep_day_index, '\tdep_time_index', dep_time_index)
        exit()

In [ ]:
windspeedKmphList = [data['windspeedKmph'] for data in flight_weather_data]
WindDirDegreeList = [data['winddirDegree'] for data in flight_weather_data]
weathercodeList = [data['weatherCode'] for data in flight_weather_data]
precipMMList = [data['precipMM'] for data in flight_weather_data]
visibilityList = [data['visibility'] for data in flight_weather_data]
pressureList = [data['pressure'] for data in flight_weather_data]
cloudcoverList = [data['cloudcover'] for data in flight_weather_data]
DewPointFList = [data['DewPointF'] for data in flight_weather_data]
WindGustKmphList = [data['WindGustKmph'] for data in flight_weather_data]
tempFList = [data['tempF'] for data in flight_weather_data]
WindChillFList = [data['WindChillF'] for data in flight_weather_data]
HumidityList = [data['humidity'] for data in flight_weather_data]
timelist = [data['time'] for data in flight_weather_data]


df1['windspeedKmph'] = windspeedKmphList
df1['winddirDegree'] = WindDirDegreeList
df1['weatherCode'] = weathercodeList 
df1['precipMM'] = precipMMList
df1['visibility'] = visibilityList 
df1['pressure'] = pressureList
df1['DewPointF'] = DewPointFList
df1['cloudcover'] = cloudcoverList
df1['WindGustKmph'] = WindGustKmphList
df1['tempF'] = tempFList
df1['WindChillF'] = WindChillFList
df1['humidity'] = HumidityList
df1['tempF'] = tempFList
df1['time'] = timelist
print(df1)


In [ ]:
df1.to_csv('file2.csv')

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [15]:
# Assume 'X' is your feature matrix and 'y' is your target variable
df2 = df1.dropna()
X = df2[["Year", "Month", "DayofMonth", "DepTime", "DepDel15", "CRSDepTime", "DepDelayMinutes", "OriginAirportID", "DestAirportID", "ArrTime", "CRSArrTime", "ArrDelayMinutes", "windspeedKmph", "winddirDegree", "weatherCode", "precipMM", "visibility", "pressure", "DewPointF", "cloudcover", "WindGustKmph", "tempF", "WindChillF", "humidity", "time"]]
y = df2 ["ArrDel15"]
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [16]:
# # Verify the shapes of the resulting datasets
# print("X_train shape:", X_train.shape)
# print("X_test shape:", X_test.shape)
# print("y_train shape:", y_train.shape)
# print("y_test shape:", y_test.shape)

model = LogisticRegression()

# Train the model using the training data
model.fit(X_train, y_train)

# Predict on the training set
y_train_pred = model.predict(X_train)

# Predict on the testing set
y_test_pred = model.predict(X_test)

# Evaluate the model's performance
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)


C:\Users\Ram\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Training Accuracy: 0.958599966567054
Testing Accuracy: 0.9585761200027668


In [18]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# Assume 'X_train', 'X_test', 'y_train', 'y_test' are your training and testing sets

In [21]:
for i in df1.columns:
    i

Unnamed: 0 <class 'str'>
FlightDate <class 'str'>
Quarter <class 'str'>
Origin <class 'str'>
Year <class 'str'>
Month <class 'str'>
DayofMonth <class 'str'>
DepTime <class 'str'>
DepDel15 <class 'str'>
CRSDepTime <class 'str'>
DepDelayMinutes <class 'str'>
OriginAirportID <class 'str'>
DestAirportID <class 'str'>
ArrTime <class 'str'>
CRSArrTime <class 'str'>
ArrDel15 <class 'str'>
ArrDelayMinutes <class 'str'>
windspeedKmph <class 'str'>
winddirDegree <class 'str'>
weatherCode <class 'str'>
precipMM <class 'str'>
visibility <class 'str'>
pressure <class 'str'>
DewPointF <class 'str'>
cloudcover <class 'str'>
WindGustKmph <class 'str'>
tempF <class 'str'>
WindChillF <class 'str'>
humidity <class 'str'>
time <class 'str'>


In [33]:
df1 = df1.dropna()
# Assume 'X' is your feature matrix and 'y' is your target variable


column_list =["Year", "Month", "DayofMonth", "DepTime", "DepDel15", "CRSDepTime", "DepDelayMinutes", "OriginAirportID", "DestAirportID", "ArrTime", "CRSArrTime","ArrDel15", "windspeedKmph", "winddirDegree", "weatherCode", "precipMM", "visibility", "pressure", "DewPointF", "cloudcover", "WindGustKmph", "tempF", "WindChillF", "humidity", "time", "ArrDelayMinutes"]
#df1[column_list] = df1[column_list].astype(float)

X = df1[["Year", "Month", "DayofMonth", "DepTime", "DepDel15", "CRSDepTime", "DepDelayMinutes", "OriginAirportID", "DestAirportID", "ArrTime", "CRSArrTime","ArrDel15", "windspeedKmph", "winddirDegree", "weatherCode", "precipMM", "visibility", "pressure", "DewPointF", "cloudcover", "WindGustKmph", "tempF", "WindChillF", "humidity", "time"]]
y = df1 [ "ArrDelayMinutes"]
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# reg = ExtraTreesRegressor(n_estimators=100, random_state=0).fit(X_train, y_train)
# reg.score(X_test, y_test)

# from sklearn.metrics import mean_squared_error, mean_absolute_error

# # Assuming 'reg' is your trained regression model

# # Predict on the testing set
# y_pred = reg.predict(X_test)

# # Calculate MSE, RMSE, and MAE
# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# mae = mean_absolute_error(y_test, y_pred)

# print("MSE:", mse)
# print("RMSE:", rmse)
# print("MAE:", mae)



# # Sample input data (X) and corresponding output data (y)


# # Create an XGBoost regressor model
# model = xgb.XGBRegressor()

# # Fit the model to the training data
# model.fit(X_train, y_train)

# # Predict the output for the test data
# y_pred = model.predict(X_test)

# # Calculate mean squared error (MSE)
# mse = mean_squared_error(y_test, y_pred)

# # Print the predicted values for the test data
# print("Predicted values:", y_pred)

# # Print the mean squared error
# print("Mean Squared Error:", mse)



def bagging_regressor(X, y, num_models, test_size=0.2):
    """
    Perform bagging ensemble for regression.

    Parameters:
        X (array-like): Input features.
        y (array-like): Target values.
        num_models (int): Number of individual models to train.
        test_size (float, optional): Proportion of the data to use for testing (default: 0.2).

    Returns:
        ensemble_pred (array-like): Predictions from the ensemble.
    """
    ensemble_pred = np.zeros_like(y)  # Array to store ensemble predictions

    for _ in range(num_models):
        # Randomly split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

        # Train your model on the training set
        model = YourRegressor()  # Replace YourRegressor with your chosen regression model
        model.fit(X_train, y_train)

        # Make predictions on the testing set
        y_pred = model.predict(X_test)

        # Add the individual predictions to the ensemble predictions
        ensemble_pred += y_pred

    # Average the predictions of all the individual models
    ensemble_pred /= num_models

    return ensemble_pred

ensemble_predictions = bagging_regressor(X, y, num_models)

# Calculate and print the MSE
mse = mean_squared_error(y, ensemble_predictions)
print(f"Mean Squared Error: {mse}")


KeyError: "['windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM', 'visibility', 'pressure', 'DewPointF', 'cloudcover', 'WindGustKmph', 'tempF', 'WindChillF', 'humidity', 'time'] not in index"